# This Notebook Creates UC functions as Tools for Agents

In [11]:
import mlflow
from databricks.connect import DatabricksSession
from dotenv import load_dotenv

spark = DatabricksSession.builder.getOrCreate()
load_dotenv('../.env')

DATABRICKS_HOST = os.getenv('host')
DATABRICKS_TOKEN = os.getenv('token')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

print(f"host: {DATABRICKS_HOST}")

host: adb-984752964297111.11.azuredatabricks.net


In [12]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()
set_uc_function_client(client)

CATALOG = 'qyu'
SCHEMA = 'dbdemos_fs_travel'
model = 'dbdemos_fs_travel_model'

In [13]:
def show_uc_tools(catalog: str, schema: str):
    """
    Show UC tools in a given catalog and schema
    """
    func_client = DatabricksFunctionClient()
    functions = func_client.list_functions(catalog=catalog, schema=schema)
    for func in functions:
        print(f"Name: {func.name} | Description: {func.comment}")

show_uc_tools(CATALOG, SCHEMA)

Name: model_asset_collector | Description: Collect ML model assets from MLFlow and save them to a UC Volume folder
Name: model_attributes_to_markdown | Description: Create ML model attributes markdown file from MLFlow and save them to a UC Volume folder


## UC Function: `model_asset_collector()`

In [16]:
sql_function = \
"""
CREATE OR REPLACE FUNCTION qyu.dbdemos_fs_travel.model_asset_collector(
    catalog STRING,
    schema STRING,
    model_name STRING
)
RETURNS STRING
COMMENT 'Collect ML model assets from MLFlow and save them to a UC Volume folder'
LANGUAGE PYTHON
ENVIRONMENT (
    dependencies = '["mlflow==2.21.3"]',
    environment_version = 'None'
)
AS $$
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_registry_uri("databricks-uc")
client = MlflowClient()
model_full_name = f"{catalog}.{schema}.{model_name}"
volume_path = f"/Volumes/{catalog}/{schema}/ml_documents"
model_version = client.get_model_version_by_alias(name=model_full_name, alias="production")
run_id = model_version.run_id

dst_path = os.path.join(volume_path, model_name)

# Check if the folder exists and create it if it does not
if not os.path.isdir(dst_path):
    try:
        os.makedirs(dst_path)
        print(f"PASS: Folder `{dst_path}` created")
    except PermissionDenied:
        print(f"FAIL: No permissions to create folder `{dst_path}`")
        raise ValueError(f"No permissions to create folder `{dst_path}`")
else:
    print(f"PASS: Folder `{dst_path}` already exists")

mlflow.artifacts.download_artifacts(artifact_uri=f"runs:/{run_id}/", dst_path=dst_path)
return dst_path
$$;
"""
sql_func_info = client.create_function(sql_function_body=sql_function)

## UC Functions: `model_attributes_to_markdown()`

In [14]:
sql_function = \
"""
CREATE OR REPLACE FUNCTION qyu.dbdemos_fs_travel.model_attributes_to_markdown(
    catalog STRING,
    schema STRING,
    model_name STRING
)
RETURNS STRING
COMMENT 'Create ML model attributes markdown file from MLFlow and save them to a UC Volume folder'
LANGUAGE PYTHON
ENVIRONMENT (
    dependencies = '["mlflow==2.21.3", "markdownify>=1.1.0", "json2html>=1.3.0"]',
    environment_version = 'None'
)
AS $$
import mlflow
import json
from json2html import json2html
from mlflow.tracking import MlflowClient

client = MlflowClient()
model_full_name = f"{catalog}.{schema}.{model_name}"
dst_path = f"/Volumes/{catalog}/{schema}/ml_documents/{model_name}"
model_version = client.get_model_version_by_alias(name=model_full_name, alias="production")
run_id = model_version.run_id
run = client.get_run(run_id)

# model properties
model_flattened_json = json.dumps({
    **run.data.params,
    **run.data.metrics,
    **run.data.tags
}, indent=4)
model_html = json2html.convert(json.loads(model_flattened_json))

# run properties
run_info_dict = {key: value for key, value in run.info.__dict__.items()}
run_info_html = json2html.convert(run_info_dict)

# data source
dataset_input = run.inputs.to_dictionary()['dataset_inputs'][0]
data_source = dataset_input['dataset']
data_source_html = json2html.convert(data_source)

# consolidated information
consolidated_md = (f"# model algorithm, model parameters, and model metrics table\n"
                f"{md(model_html)}\n"
                f"# model run information table\n"
                f"{md(run_info_html)}\n"
                f"# data source table\n"
                f"{md(data_source_html)}")

with open(f"{dst_path}/model_attribute_tables.md", "w") as file:
    print(f"writing model attributes tables to {dst_path}/model_attribute_tables.md")
    file.write(consolidated_information)
return dst_path
$$;
"""
sql_func_info = client.create_function(sql_function_body=sql_function)